In [ ]:
# --- Cell 0: Interactive File Setup ---
%matplotlib widget

from pathlib import Path

import ipywidgets as widgets
from ipyfilechooser import FileChooser

import opym

print("1. Select your base OME-TIF file (e.g., ...Pos0.ome.tif).")
print("2. Select the Timepoint (T) you want to use for the MIP.")
print("3. Run Cell 1 to calculate the MIP.")

# --- Create the file chooser ---
file_chooser = FileChooser(
    path="/mmfs2/scratch/SDSMT.LOCAL/bscott/DataUpload",
    filter_pattern="*.ome.tif",
    title="<b>Select Base OME-TIF File:</b>",
)

# --- Slider for Timepoint selection ---
t_slider = widgets.IntSlider(
    description="Timepoint (T):",
    min=0,
    max=199,  # Will be updated by Cell 1
    step=1,
    value=0,
)

display(file_chooser, t_slider)

In [ ]:
# --- Cell 1: Create the MIPs ---
# This cell now calls the `create_mip` function from opym

try:
    file_to_inspect = Path(file_chooser.value)
    t_index = t_slider.value

    # Call the refactored function
    mip_data, vmin, vmax, lazy_data, t_max = opym.create_mip(file_to_inspect, t_index)

    # Update the slider in Cell 0 with the correct max T
    t_slider.max = t_max
    t_slider.value = t_index

    print("\nYou can now run Cell 2 to select ROIs.")

except FileNotFoundError:
    print("❌ ERROR: File not found. Please select a file in Cell 0 and re-run.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 2: Select ROIs Interactively ---
# This cell uses the new ROISelector class from opym

try:
    # This will display the plot and wait for you to draw two ROIs
    selector = opym.interactive_roi_selector(mip_data, vmin, vmax)
except NameError:
    print("❌ ERROR: 'mip_data' not found. Please run Cell 1 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 3: Get ROIs and Auto-Align ---
# This cell gets the ROIs and calls the new validation/alignment function

try:
    # This call will block until two ROIs are drawn in the plot from Cell 2
    unaligned_rois = selector.get_rois()

    # --- NEW: Call the new processing function from opym ---
    # This function handles validation, alignment, and None assignment
    top_roi, bottom_roi = opym.process_rois_from_selector(
        mip_data,
        unaligned_rois,
        valid_threshold=1.0,
    )
    # --- END NEW ---

    if top_roi or bottom_roi:
        print("\nRun Cell 4 to visualize.")
    else:
        print("  Re-run Cell 2 to try again.")

except NameError:
    print(
        "❌ ERROR: 'selector' or 'mip_data' not found. Please run Cells 1 and 2 first."
    )
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 4: Visualize Final Alignment ---
# This cell calls the new visualization function from opym

from typing import cast

try:
    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs found. Please re-run Cell 2 and 3.")
    elif top_roi is None:
        print("ℹ️ Only Bottom ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know bottom_roi is not None here
        valid_bottom = cast(tuple[slice, slice], bottom_roi)
        opym.visualize_alignment(mip_data, valid_bottom, valid_bottom, vmin, vmax)
    elif bottom_roi is None:
        print("ℹ️ Only Top ROI is valid. Skipping visualization.")
        # Cast to satisfy Pylance: we know top_roi is not None here
        valid_top = cast(tuple[slice, slice], top_roi)
        opym.visualize_alignment(mip_data, valid_top, valid_top, vmin, vmax)
    else:
        # Both are valid, show alignment
        opym.visualize_alignment(mip_data, top_roi, bottom_roi, vmin, vmax)

    print("\nRun Cell 5 to save these ROIs to your log file.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run Cells 1, 2, and 3 first.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 5: Save ROIs to Log File ---
# This cell uses the save_rois_to_log function from your package

try:
    # This is the central log file used by the CLI, as per the README.
    cli_log_file = Path("opm_roi_log.json")

    if top_roi is None and bottom_roi is None:
        print("❌ ERROR: No valid ROIs to save. Please re-run Cells 2 & 3.")
    else:
        opym.save_rois_to_log(
            log_file=cli_log_file,
            base_file=file_to_inspect,
            top_roi=top_roi,  # Can be None
            bottom_roi=bottom_roi,  # Can be None
        )
        print(f"✅ ROIs saved to {cli_log_file.name}")
        print("You can now use these ROIs for batch processing via the CLI,")
        print("or run Cell 6 to process this file immediately.")

except NameError:
    print("❌ ERROR: Required variables not found. Please run all previous cells.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
# --- Cell 6: Submit Cropping Job & Save Settings Sidecar ---
import ipywidgets as widgets
from IPython.display import display
from pathlib import Path
from typing import cast
import opym.ui
import opym.dataloader
import opym.petakit

# 1. Validate Globals
if "file_to_inspect" not in globals():
    print("❌ ERROR: Run previous cells first.")
else:
    # 2. Setup UI
    fpath = cast(Path, globals()["file_to_inspect"])
    n_ch = opym.dataloader.get_channel_count(fpath)
    
    ui_accordion, channel_checks, fmt_w, rot_w = opym.ui.create_crop_settings_ui(n_channels=n_ch)
    
    run_btn = widgets.Button(description="Submit Crop", button_style="primary", icon="rocket")
    status_out = widgets.Output()

    def on_submit(b):
        global processing_output_dir
        with status_out:
            status_out.clear_output()
            
            # Re-fetch globals safely inside callback
            _file = globals().get('file_to_inspect')
            _top = globals().get('top_roi')
            _bot = globals().get('bottom_roi')

            if _file is None or _top is None or _bot is None:
                print("❌ ERROR: Missing File or ROI data.")
                return

            # Check Channels
            selected_ch = sorted([cid for cid, cb in channel_checks.items() if cb.value])
            if not selected_ch:
                print("⚠️ No channels selected.")
                return

            # Pylance Fix: Ensure dropdown value is not None before accessing attributes
            if fmt_w.value is None:
                print("❌ ERROR: Output format not selected.")
                return
            
            # Now safe to access .value (The enum member's value string)
            fmt_string = fmt_w.value.value

            print("🚀 Submitting to Petakit...")
            try:
                # One-liner execution
                job_path, out_dir = opym.petakit.submit_crop_and_save_sidecar(
                    file_path=_file,
                    top_roi=_top,
                    bottom_roi=_bot,
                    channels=selected_ch,
                    output_format=fmt_string,
                    rotate=rot_w.value
                )
                
                # Update Global State
                processing_output_dir = out_dir
                
                print(f"✅ Job Ticket: {job_path.name}")
                print(f"📂 Output Dir: {processing_output_dir.name}")
                
            except Exception as e:
                print(f"❌ Error: {e}")

    run_btn.on_click(on_submit)
    
    display(widgets.VBox([
        widgets.HTML("<h3>Step 6: Configure & Submit Crop</h3>"),
        ui_accordion,
        run_btn,
        status_out
    ]))

In [ ]:
# --- Cell 7: Submit Deskew Job ---
import json
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display
import opym.petakit
import opym.metadata
import opym.ui

# 1. Validation
_global_target = globals().get('processing_output_dir')
if not isinstance(_global_target, Path):
    print("⚠️ Output directory missing. Run Cell 6 first.")
else:
    target_dir = _global_target
    
    # 2. Auto-Detect Z
    meta_file = target_dir.parent / "AcqSettings.txt"
    detected_z = 1.0
    if meta_file.exists():
        try: detected_z = opym.metadata.parse_z_step(meta_file, 1.0)
        except: pass

    # 3. Create UI
    w_z, w_angle, w_px, btn, status, log = opym.ui.create_deskew_ui(detected_z=detected_z)
    
    def on_deskew_click(b):
        with log:
            log.clear_output()
            try:
                print(f"🚀 Submitting Deskew: {target_dir.name}...")
                ticket = opym.petakit.submit_remote_deskew_job(
                    input_target=target_dir,
                    z_step_um=w_z.value,
                    xy_pixel_size=w_px.value,
                    sheet_angle_deg=w_angle.value,
                    deskew=True, rotate=True
                )
                
                # Update Sidecar
                sidecar = target_dir / "petakit_settings.json"
                settings = {}
                if sidecar.exists():
                    with open(sidecar, "r") as f: settings = json.load(f)
                
                settings["deskew"] = {
                    "z_step_um": w_z.value,
                    "sheet_angle_deg": w_angle.value,
                    "xy_pixel_size": w_px.value
                }
                
                with open(sidecar, "w") as f: json.dump(settings, f, indent=4)
                
                print(f"✅ Ticket: {ticket.name}")
                status.value = "⏳ Running..."
                opym.petakit.monitor_job_background(ticket, status)
                
            except Exception as e:
                print(f"❌ Error: {e}")
                status.value = "❌ Error"

    btn.on_click(on_deskew_click)
    
    display(widgets.VBox([
        widgets.HTML("<h3>Step 7: Deskew Parameters</h3>"),
        widgets.HBox([w_z, w_angle, w_px]),
        widgets.HBox([btn, status]),
        log
    ]))

In [ ]:
# --- Cell 8: Load Final DSR Data ---
import ipywidgets as widgets
from IPython.display import display
import opym.dataloader

btn_load = widgets.Button(description="Load DSR Data", button_style="primary", icon="folder-open")
out_load = widgets.Output()

def on_load_click(b):
    global viewer_params
    with out_load:
        out_load.clear_output()
        try:
            # 1. Find Data (using new helper)
            _global_dir = globals().get("processing_output_dir")
            dsr_path = opym.dataloader.find_dsr_directory(_global_dir)
            print(f"📂 Loading from: {dsr_path}")

            # 2. Load
            loader_res = opym.load_tiff_series(dsr_path)
            
            # 3. Unpack & Store Global Params
            (get_stack, t_min, t_max, c_min, c_max, z_max, y, x, base) = loader_res
            viewer_params = (get_stack, t_max, z_max, c_max, y, x)
            
            print(f"✅ Loaded: {base}")
            print(f"   Shape: T={t_max+1}, Z={z_max+1}, C={c_max+1}, Y={y}, X={x}")
            print("Run Cell 9/10 to view.")
            
        except Exception as e:
            print(f"❌ Error: {e}")

btn_load.on_click(on_load_click)
display(widgets.VBox([btn_load, out_load]))

In [ ]:
# --- Cell 9: Launch the single-channel viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.single_channel_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 10: Launch the composite viewer ---
try:
    if viewer_params is None:
        raise NameError
    opym.composite_viewer(*viewer_params)
except NameError:
    print("❌ ERROR: 'viewer_params' not found. Please run Cell 8 to load data first.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")

In [ ]:
# --- Cell 11: Batch Process using "Gold Standard" Sidecar ---
import json
from pathlib import Path

import ipywidgets as widgets
import opym.batch
from IPython.display import display

# 1. Validation & Load
_global_output_dir = globals().get("processing_output_dir")
if not isinstance(_global_output_dir, Path):
    print("⚠️ 'processing_output_dir' missing. Run Cell 6 first.")
    # Stop execution manually if this fails
else:
    TEMPLATE_FOLDER = _global_output_dir
    sidecar_file = TEMPLATE_FOLDER / "petakit_settings.json"

    if not sidecar_file.exists():
        print(f"❌ ERROR: Sidecar missing: {sidecar_file}")
    else:
        with open(sidecar_file, "r") as f:
            settings = json.load(f)

        # 2. Identify Files
        data_root = TEMPLATE_FOLDER.parent
        all_files = sorted(list(data_root.glob("**/*_MMStack_Pos0.ome.tif")))
        source_name = settings.get("source_file", "")
        files_to_process = [f for f in all_files if f.name != source_name]

        if not files_to_process:
            print("🎉 No other files found.")
        else:
            # 3. Setup UI
            print(f"🚀 Batch Processing {len(files_to_process)} files...")
            
            pbar = widgets.IntProgress(
                value=0, 
                max=len(files_to_process), 
                description="Progress:", 
                layout=widgets.Layout(width="100%"),
                style={"bar_color": "#00d1b2"}
            )
            status = widgets.Label("Initializing...")
            log = widgets.Output(layout={"border": "1px solid #ddd", "height": "200px", "overflow_y": "scroll"})
            
            display(widgets.VBox([status, pbar, log]))

            # 4. Run Batch Logic
            opym.batch.run_batch_cropping(
                template_folder=TEMPLATE_FOLDER,
                file_list=files_to_process,
                settings=settings,
                log_output=log,
                progress_bar=pbar,
                status_label=status
            )